Dipendenze

In [ ]:
%pip install nltk
%pip install datasets
%pip install transformers[torch]
%pip install tokenizers
%pip install evaluate
%pip install rouge_score
%pip install sentencepiece
%pip install huggingface_hub
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Generazione risposta del modello con singola stringa

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import re

# Questa funzione si occupa di ottenere le entità estratte dal modello e di organizzarle in un dizionario

def extract_entities_with_categories_from_prediction(sentence):
  pattern = re.compile(r'(\w+):\s*([^;]+)')
  matches = pattern.findall(sentence)
  result_dict = {}
  for match in matches:
      key = match[0].strip()
      values = match[1].strip()
      if(key in result_dict.keys()):
        result_dict[key].append(values)
      else:
        result_dict[key] = []
        result_dict[key].append(values)
  return result_dict

# Funzione per generare la risposta ad una domanda, restituisce un dizionario con le entità estratte dal modello e i rispettivi tipi
def generate_question(question,path_model):
  prefix = "NER: "
  last_checkpoint = path_model
  finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(last_checkpoint, device_map="auto")
  tokenizer = AutoTokenizer.from_pretrained(last_checkpoint)
  inputs = f"{prefix}: {question}"
  inputs = tokenizer(inputs, return_tensors="pt").to("cuda")
  outputs = finetuned_model.generate(**inputs,max_new_tokens= 80,do_sample = False)
  answer = tokenizer.decode(outputs[0],skip_special_tokens = True)
  return extract_entities_with_categories_from_prediction(answer)

path_model = "Andro9669/flan-t5-ner"
question = "An entity-oriented approach to restricted-domain parsing is proposed "
print(generate_question(question, path_model))

Generazione risposte in file JSON del test set di SCIERC

In [ ]:
import json
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Path del checkpoint del modello fine tunato, del file json dei risultati e del dataset
path_model = "Andro9669/flan-t5-ner"
path_result = "/content/drive/MyDrive/Tesi/result/result_flan_large_no_diff_10.json"
path_dataset = "/content/drive/MyDrive/Tesi/dataset/scierc_test_no_diff.json"


finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(path_model,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(path_model)

prefix = "NER: "
# Per ogni domanda del dataset, genero la risposta e la salvo nel file json dei risultati
with open(path_dataset) as f:
    for line in f:
        data = json.loads(line)
        question = data["input"]
        inputs = f"{prefix}: {question}"
        inputs = tokenizer(inputs, return_tensors="pt").to("cuda")
        outputs = finetuned_model.generate(**inputs,max_new_tokens= 80,do_sample = False)
        answer = tokenizer.decode(outputs[0],skip_special_tokens = True)
        print(answer)
        with open(path_result, 'a') as outfile:
          json_object = {"prediction": answer}
          json.dump(json_object, outfile)
          outfile.write('\n')